In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.0.0-alpha0


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [77]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [71]:
# create a model
model = create_model()

# show a model
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_24 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoint_path = "Ex1102/1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [7]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 verbose=1)

In [8]:
# Train the model with the new callback
# Pass callback to training
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 800/1000 [=======================>......] - ETA: 0s - loss: 1.2515 - accuracy: 0.6463
Epoch 00001: val_loss improved from inf to 0.66816, saving model to Ex1102/1.ckpt
1000/1000 [==============================] - 1s 822us/sample - loss: 1.1570 - accuracy: 0.6720 - val_loss: 0.6682 - val_accuracy: 0.8020
Epoch 2/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.4255 - accuracy: 0.8888
Epoch 00002: val_loss improved from 0.66816 to 0.53106, saving model to Ex1102/1.ckpt
1000/1000 [==============================] - 0s 245us/sample - loss: 0.4190 - accuracy: 0.8890 - val_loss: 0.5311 - val_accuracy: 0.8380
Epoch 3/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.2644 - accuracy: 0.9388
Epoch 00003: val_loss improved from 0.53106 to 0.45473, saving model to Ex1102/1.ckpt
1000/1000 [==============================] - 0s 253us/sample - loss: 0.2816 - accuracy: 0.9290 - val_loss: 0.4547 - val_accuracy: 

In [9]:
ls {checkpoint_dir}

1.ckpt.data-00000-of-00001  1.ckpt.index  checkpoint


In [10]:
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)

1000/1000 - 0s - loss: 0.4035 - accuracy: 0.8690


In [11]:
# create a model
model = create_model()

# evaluate model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 2.4005 - accuracy: 0.1360
Untrained model, accuracy: 13.60%


In [12]:
model.load_weights(checkpoint_path)

# reevaluate model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.3939 - accuracy: 0.8620
Restored model, accuracy: 86.20%


In [13]:
# insert epoch into file name(use `str.format`)
checkpoint_path = "Ex1102/2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback，save model weight for every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_best_only=True,
    save_weights_only=True,
    period=5)

model = create_model()

# use 'checkpoint_path' to save weight
model.save_weights(checkpoint_path.format(epoch=0))

# train
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: val_loss improved from inf to 0.44555, saving model to Ex1102/2/cp-0005.ckpt

Epoch 00010: val_loss improved from 0.44555 to 0.42472, saving model to Ex1102/2/cp-0010.ckpt

Epoch 00015: val_loss improved from 0.42472 to 0.41288, saving model to Ex1102/2/cp-0015.ckpt

Epoch 00020: val_loss did not improve from 0.41288

Epoch 00025: val_loss did not improve from 0.41288

Epoch 00030: val_loss did not improve from 0.41288

Epoch 00035: val_loss did not improve from 0.41288

Epoch 00040: val_loss did not improve from 0.41288

Epoch 00045: val_loss did not improve from 0.41288

Epoch 00050: val_loss did not improve from 0.41288


In [14]:
ls {checkpoint_dir}

checkpoint                        cp-0010.ckpt.data-00000-of-00001
cp-0000.ckpt.data-00000-of-00001  cp-0010.ckpt.index
cp-0000.ckpt.index                cp-0015.ckpt.data-00000-of-00001
cp-0005.ckpt.data-00000-of-00001  cp-0015.ckpt.index
cp-0005.ckpt.index


In [15]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'Ex1102/2/cp-0015.ckpt'

In [41]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4129 - accuracy: 0.8740
Restored model, accuracy: 87.40%


In [42]:
model.save_weights('./Ex1102/saveweights')

model = create_model()

model.load_weights('./Ex1102/saveweights')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4129 - accuracy: 0.8740
Restored model, accuracy: 87.40%


In [78]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p ./Ex1102/saved_model
model.save('./Ex1102/saved_model/my_model') 

Epoch 1/5
1000/1000 [==============================] - 0s 269us/sample - loss: 1.1683 - accuracy: 0.6650
Epoch 2/5
1000/1000 [==============================] - 0s 149us/sample - loss: 0.4484 - accuracy: 0.8650
Epoch 3/5
1000/1000 [==============================] - 0s 150us/sample - loss: 0.3107 - accuracy: 0.9090
Epoch 4/5
1000/1000 [==============================] - 0s 149us/sample - loss: 0.1983 - accuracy: 0.9550
Epoch 5/5
1000/1000 [==============================] - 0s 154us/sample - loss: 0.1721 - accuracy: 0.9590


In [67]:
# my_model folder
!ls ./Ex1102/saved_model

my_model  my_model.h5  my_model.model


In [68]:
# including assets，saved_model.pb，and variables
!ls ./Ex1102/saved_model/my_model

./Ex1102/saved_model/my_model


In [69]:
new_model = keras.models.load_model('./Ex1102/saved_model/my_model')

new_model.summary()

ValueError: Unknown entry in loss dictionary: class_name. Only expected following keys: ['dense_47']

In [57]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

NameError: name 'new_model' is not defined

In [60]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 将整个模型保存为 HDF5 文件。
# '.h5' 扩展名指示应将模型保存到 HDF5。
model.save('./Ex1102/saved_model/my_model.h5') 

Epoch 1/5
1000/1000 [==============================] - 0s 237us/sample - loss: 1.1754 - accuracy: 0.6720
Epoch 2/5
1000/1000 [==============================] - 0s 142us/sample - loss: 0.4321 - accuracy: 0.8860
Epoch 3/5
1000/1000 [==============================] - 0s 142us/sample - loss: 0.2805 - accuracy: 0.9310
Epoch 4/5
1000/1000 [==============================] - 0s 138us/sample - loss: 0.2056 - accuracy: 0.9570
Epoch 5/5
1000/1000 [==============================] - 0s 144us/sample - loss: 0.1552 - accuracy: 0.9620


In [61]:
# 重新创建完全相同的模型，包括其权重和优化程序
new_model = tf.keras.models.load_model('./Ex1102/saved_model/my_model.h5')

# 显示网络结构
new_model.summary()

ValueError: Unknown entry in loss dictionary: class_name. Only expected following keys: ['dense_43']

In [62]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

NameError: name 'new_model' is not defined